# Getting set up

## Setting up the notebook

In [2]:
### Installing dependencies
!pip install openai

!apt-get update
!apt-get install -y iverilog

Hit:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:2 https://cli.github.com/packages stable InRelease
Hit:3 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:5 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:6 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:7 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Reading package lists... Done
W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
iverilog is already the newest version (11.0-1.1).
0 upgraded, 0 newly installed, 0 to remove 

## Use LLM to generate Verilog code from natural language description

### Example 1: binary_to_bcd

In [3]:
! mkdir -p binary_to_bcd
! cd binary_to_bcd && curl -O https://raw.githubusercontent.com/AKHASHHH/LLM4CHIPDESIGN/refs/heads/main/ChipChat/Test%20Bench/binary_to_bcd_tb.v

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  1939  100  1939    0     0   9251      0 --:--:-- --:--:-- --:--:--  9277


In [4]:
verilog_generation_prompt = '''
I am trying to create a Verilog model binary_to_bcd_converter for a binary to binary-coded-decimal converter. It must meet the following specifications:
Inputs:

binary_input (5 bits)

enable (1 bit) – when high, the output is updated; when low, the output holds its previous value

clk (1 bit) – rising edge used to register the BCD output
Outputs:

bcd_output (8 bits: 4 bits for the 10's place and 4 bits for the 1's place)
Behavior:

Compute the BCD representation of the 5‑bit binary input (values 0–31).

On each rising edge of clk, if enable is high, update bcd_output with the new BCD value.

If enable is low, keep bcd_output unchanged.

How would I write a design that meets these specifications?

module binary_to_bcd_converter (
    input clk,
    input enable,
    input [4:0] binary_input,
    output reg [7:0] bcd_output
);

reg [4:0] stored_binary;

always @(posedge clk) begin
    if (enable) begin
        stored_binary <= binary_input;

        // Binary to BCD conversion using double dabble algorithm
        // Convert 5-bit binary to BCD (tens and units)
        case (stored_binary)
            5'd0:   bcd_output <= 8'b00000000; // 0
            5'd1:   bcd_output <= 8'b00000001; // 1
            5'd2:   bcd_output <= 8'b00000010; // 2
            5'd3:   bcd_output <= 8'b00000011; // 3
            5'd4:   bcd_output <= 8'b00000100; // 4
            5'd5:   bcd_output <= 8'b00000101; // 5
            5'd6:   bcd_output <= 8'b00000110; // 6
            5'd7:   bcd_output <= 8'b00000111; // 7
            5'd8:   bcd_output <= 8'b00001000; // 8
            5'd9:   bcd_output <= 8'b00001001; // 9
            5'd10:  bcd_output <= 8'b00010000; // 10
            5'd11:  bcd_output <= 8'b00010001; // 11
            5'd12:  bcd_output <= 8'b00010010; // 12
            5'd13:  bcd_output <= 8'b00010011; // 13
            5'd14:  bcd_output <= 8'b00010100; // 14
            5'd15:  bcd_output <= 8'b00010101; // 15
            5'd16:  bcd_output <= 8'b00010110; // 16
            5'd17:  bcd_output <= 8'b00010111; // 17
            5'd18:  bcd_output <= 8'b00011000; // 18
            5'd19:  bcd_output <= 8'b00011001; // 19
            5'd20:  bcd_output <= 8'b00100000; // 20
            5'd21:  bcd_output <= 8'b00100001; // 21
            5'd22:  bcd_output <= 8'b00100010; // 22
            5'd23:  bcd_output <= 8'b00100011; // 23
            5'd24:  bcd_output <= 8'b00100100; // 24
            5'd25:  bcd_output <= 8'b00100101; // 25
            5'd26:  bcd_output <= 8'b00100110; // 26
            5'd27:  bcd_output <= 8'b00100111; // 27
            5'd28:  bcd_output <= 8'b00101000; // 28
            5'd29:  bcd_output <= 8'b00101001; // 29
            5'd30:  bcd_output <= 8'b00110000; // 30
            5'd31:  bcd_output <= 8'b00110001; // 31
            default: bcd_output <= 8'b00000000;
        endcase
    end
end

endmodule

clear the below error

Testing Binary-to-BCD Converter with enable...
VCD info: dumpfile my_design.vcd opened for output.
Error (enable=1): bin=1, expected BCD=8'b1, got=8'b0
'''

Now you can use the openai library to generate text from your prompt.

Before using LLMs for Verilog generation, you have to first get the api-key:

openai llms: https://platform.openai.com/api-keys
deepseek llms: https://platform.deepseek.com/sign_in
free llms provided by NVIDIA: https://build.nvidia.com/explore/discover

In [5]:
from openai import OpenAI
import os
client = OpenAI(base_url="https://openrouter.ai/api/v1",
    api_key = "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"
)

completion = client.chat.completions.create(
  model = "arcee-ai/trinity-large-preview:free",
  messages=[{"role":"user","content":verilog_generation_prompt}],
  max_tokens=129000, # limits the maximum number of tokens (words or pieces of words) that the model will generate in its response.
  stream=False
)

print(completion.choices[0].message.content)

The error occurs because you're updating `stored_binary` before using it in the case statement. When `enable` is high, you assign `stored_binary <= binary_input` and then immediately use `stored_binary` in the case statement—but this reads the **old value** of `stored_binary` because the assignment hasn't taken effect yet (it's a non-blocking assignment that updates on the next clock edge).

You have two options to fix this:

**Option 1: Use binary_input directly (simpler)**

```verilog
module binary_to_bcd_converter (
    input clk,
    input enable,
    input [4:0] binary_input,
    output reg [7:0] bcd_output
);

always @(posedge clk) begin
    if (enable) begin
        case (binary_input)
            5'd0:   bcd_output <= 8'b00000000; // 0
            5'd1:   bcd_output <= 8'b00000001; // 1
            5'd2:   bcd_output <= 8'b00000010; // 2
            5'd3:   bcd_output <= 8'b00000011; // 3
            5'd4:   bcd_output <= 8'b00000100; // 4
            5'd5:   bcd_output <= 8'b0

Next, extract the generated verilog code from LLM response.

In [6]:
output_verilog_code = '''
module binary_to_bcd_converter (
    input clk,
    input enable,
    input [4:0] binary_input,
    output reg [7:0] bcd_output
);

always @(posedge clk) begin
    if (enable) begin
        case (binary_input)
            5'd0:   bcd_output <= 8'b00000000; // 0
            5'd1:   bcd_output <= 8'b00000001; // 1
            5'd2:   bcd_output <= 8'b00000010; // 2
            5'd3:   bcd_output <= 8'b00000011; // 3
            5'd4:   bcd_output <= 8'b00000100; // 4
            5'd5:   bcd_output <= 8'b00000101; // 5
            5'd6:   bcd_output <= 8'b00000110; // 6
            5'd7:   bcd_output <= 8'b00000111; // 7
            5'd8:   bcd_output <= 8'b00001000; // 8
            5'd9:   bcd_output <= 8'b00001001; // 9
            5'd10:  bcd_output <= 8'b00010000; // 10
            5'd11:  bcd_output <= 8'b00010001; // 11
            5'd12:  bcd_output <= 8'b00010010; // 12
            5'd13:  bcd_output <= 8'b00010011; // 13
            5'd14:  bcd_output <= 8'b00010100; // 14
            5'd15:  bcd_output <= 8'b00010101; // 15
            5'd16:  bcd_output <= 8'b00010110; // 16
            5'd17:  bcd_output <= 8'b00010111; // 17
            5'd18:  bcd_output <= 8'b00011000; // 18
            5'd19:  bcd_output <= 8'b00011001; // 19
            5'd20:  bcd_output <= 8'b00100000; // 20
            5'd21:  bcd_output <= 8'b00100001; // 21
            5'd22:  bcd_output <= 8'b00100010; // 22
            5'd23:  bcd_output <= 8'b00100011; // 23
            5'd24:  bcd_output <= 8'b00100100; // 24
            5'd25:  bcd_output <= 8'b00100101; // 25
            5'd26:  bcd_output <= 8'b00100110; // 26
            5'd27:  bcd_output <= 8'b00100111; // 27
            5'd28:  bcd_output <= 8'b00101000; // 28
            5'd29:  bcd_output <= 8'b00101001; // 29
            5'd30:  bcd_output <= 8'b00110000; // 30
            5'd31:  bcd_output <= 8'b00110001; // 31
            default: bcd_output <= 8'b00000000;
        endcase
    end
end

endmodule
'''
filename = "binary_to_bcd/binary_to_bcd.v"
# Write the extracted Verilog code to the file
with open(filename, "w") as f:
    f.write(output_verilog_code)

Use iverilog to verify the correctness of LLM generated verilog

In [7]:
!cd binary_to_bcd/ && iverilog -g2012 -o binary_to_bcd.vvp binary_to_bcd.v binary_to_bcd_tb.v && vvp binary_to_bcd.vvp

Testing Binary-to-BCD Converter with enable...
VCD info: dumpfile my_design.vcd opened for output.
All test cases passed with enable behavior!
